In [1]:
import gym
import random
from collections import deque

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers


In [12]:
#needed for visualizing on the display
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

ModuleNotFoundError: No module named 'tqdm'

In [4]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 1e3
        self._build_model()
        
    def _build_model(self):
        n = 24
        model = keras.Sequential()
        model.add(layers.Dense(n, input_dim=self.state_size, activation='relu'))
        model.add(layers.Dense(n, activation='relu'))
        model.add(layers.Dense(self.action_size, activation='linear'))
        model.compile(loss="mse", optimizer= optimizers.Adam(lr=self.learning_rate))
        self.model = model
        
        
    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            act_values = self.model.predict(state)
            print(act_values)
            return np.argmax(act_values[0])
        
        
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action ,reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                         np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose =0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [5]:
 # initialize gym environment and the agent


In [6]:
done = False
batch_size = 32

In [7]:
EPISODES = 1000

In [8]:
def render(env):
    plt.imshow(env.render(mode='rgb_array'))
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [11]:

env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, "/tmp/video")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1,state_size])
    for time in range(500):
        #show_state(img, env, time)
        #render(env)
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -1
        next_state = np.reshape(next_state, [1,state_size])
        agent.memorize(state, action ,reward, next_state, done)
        state = next_state
        if done:
            print(f"episode {e}/{EPISODES}, score: {time}, e: {agent.epsilon:.2}")
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode 0/1000, score: 11, e: 1.0
episode 1/1000, score: 19, e: 1.0
episode 2/1000, score: 16, e: 0.92
[[nan nan]]


KeyboardInterrupt: 